# EDS 213: World Bank & Energy Data Cleaning

Author: Leilanie Rubinstein

Date: May 15, 2025

*Questions:* 

1. **Which countries in Africa will be most affected by trade stagnation (ie highest cost of electricity per unit)?**
2. **Is there a relationship between fuel consumption/production type and degree of trade stagnation?**

In [2]:
# Load libraries
import os
import re

import matplotlib.pyplot as plt
import numpy as np
import openpyxl
import pandas as pd
import seaborn as sns

### Import World Bank IDA data

https://databank.worldbank.org/source/ida-results-measurement-system,-tier-i-database-%E2%80%93-wdi/Series/EG.ELC.ACCS.ZS

In [3]:
ida_data = pd.read_csv("data/116b125a-8cee-4efc-afef-1b6e33d96d3b_Data.csv")
ida_data

,Country Name,Country Code,Series Name,Series Code,1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],...,2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],LAV [YR20080],MRV [YR19950],MRV1 [YR20006],MRV2 [YR20081]
0,001. Benin_SBEE,BEN_SBEE,Average residential connection charge (LCU/con...,AFELE118,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
1,001. Benin_SBEE,BEN_SBEE,Average residential connection charge (USD/con...,AFELE119,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
2,001. Benin_SBEE,BEN_SBEE,"Collection rate, implicit: ratio of unit reven...",AFELE154,..,..,..,..,..,..,...,..,100,..,..,..,..,100,..,..,100
3,001. Benin_SBEE,BEN_SBEE,"Collection rate, reported by utility (% of bil...",AFELE126,..,..,..,..,..,..,...,..,..,..,..,..,..,..,..,..,..
4,001. Benin_SBEE,BEN_SBEE,"Cost of electricity per unit, capital, histori...",AFELE111,..,..,..,..,..,..,...,..,11.5822823192855,..,..,..,..,11.5822823192855,..,..,11.5822823192855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3123,Data from database: Africa Infrastructure: Ele...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Extract series names
ida_data["Series Name"].unique()

array(['Average residential connection charge (LCU/connection)',
       'Average residential connection charge (USD/connection)',
       'Collection rate, implicit: ratio of unit revenue collected to tariff (%)',
       'Collection rate, reported by utility (% of billing)',
       'Cost of electricity per unit, capital, historical (US cents/kWh)',
       'Cost of electricity per unit, LRMC, optimal trade scenario (US cents/kWh)',
       'Cost of electricity per unit, LRMC, trade stagnation scenario (US cents/kWh)',
       'Cost of electricity per unit, operating, historical (US cents/kWh)',
       'Cost of electricity per unit, total, historical (US cents/kWh)',
       'Cost recovery ratio, historical (%)',
       'Cost recovery ratio, LRMC (%)',
       'Customers, all voltage levels (number)',
       'Customers, high and medium voltage (% of total number of customers)',
       'Customers, high and medium voltage (number)',
       'Customers, high voltage (number)',
       'Customers, 

#### Variables of interest:

- "Cost of electricity per unit, LRMC, trade stagnation scenario (US cents/kWh)"
- "Cost of electricity per unit, LRMC, optimal trade scenario (US cents/kWh)"
- "Cost of electricity per unit, total, historical (US cents/kWh)"

"LRMC" = Long Run Marginal Cost

In [5]:
# Define the scenarios and their names
scenarios = {
    "trade_stag": "Cost of electricity per unit, LRMC, trade stagnation scenario (US cents/kWh)",
    "trade_optim": "Cost of electricity per unit, LRMC, optimal trade scenario (US cents/kWh)",
    "tot_historical": "Cost of electricity per unit, total, historical (US cents/kWh)",
}

# Define columns to keep
column_mapping = {
    "Country Name": "utility_code",
    "LAV [YR20080]": "lav_2008",
    "MRV2 [YR20081]": "mrv2_2008",
}

# Extract all scenarios in one loop with renamed columns
results = {}
for key, series_name in scenarios.items():
    # Extract the data for the current scenario
    df = ida_data[ida_data["Series Name"] == series_name].filter(
        items=list(column_mapping.keys()), axis=1
    )

    # Replace ".." with NaN
    df = df.replace("..", float("nan"))

    # Rename the columns
    df = df.rename(columns=column_mapping)

    # Convert numeric columns to float
    for col in ["lav_2008", "mrv2_2008"]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # Extract country names from utility_code
    df["country_name"] = df["utility_code"].apply(
        lambda x: re.sub(r"^\d+\.\s", "", x).split("_")[0]
        if isinstance(x, str)
        else None
    )

    # Drop rows where all values are NA
    df = df.dropna(how="all")

    # Reset index
    df = df.reset_index(drop=True)

    # Store in results dictionary
    results[key] = df

# Access scenario by name
trade_stag = results["trade_stag"]
trade_optim = results["trade_optim"]
tot_historical = results["tot_historical"]
tot_historical

,utility_code,lav_2008,mrv2_2008,country_name
0,001. Benin_SBEE,13.725913,13.725913,Benin
1,002. Burkina Faso_Sonabel,23.557524,23.557524,Burkina Faso
2,003. Cameroon_AES SONEL,13.979235,13.979235,Cameroon
3,004. Cape Verde_Electra,23.342074,23.342074,Cape Verde
4,005. Chad_STEE,83.604324,83.604324,Chad
5,"006. Congo, Dem. Rep._SNEL",6.754000,6.754000,"Congo, Dem. Rep."
6,007. CongoRep_SNE,0.120000,0.120000,CongoRep
7,008. Cote d'Ivoire_CIE,NaN,NaN,Cote d'Ivoire
8,009. Ethiopia_EEPCO,8.814480,8.814480,Ethiopia
9,010. Ghana_ECG,13.244049,13.244049,Ghana


#### Write to CSV

In [6]:
trade_stag.to_csv(os.path.join("energy_data", "ida_trade_stag.csv"))
trade_optim.to_csv(os.path.join("energy_data", "ida_trade_optim.csv"))
tot_historical.to_csv(os.path.join("energy_data", "ida_tot_historical.csv"))

### Import Energy Institute World Energy Data

https://www.energyinst.org/statistical-review/resources-and-data-downloads

In [7]:
energy_inst = pd.read_excel(
    "data/Statistical Review of World Energy Data.xlsx", sheet_name=None
)

In [8]:
print(energy_inst.keys())

# Extract the data from the first sheet
energy_data_0 = energy_inst["Contents"]
energy_data_0.values

dict_keys(['Contents', 'Primary energy cons - EJ', 'PE Cons by fuel EJ', 'PE cons per capita GJ', 'Carbon Dioxide from Energy', 'CO2 From Flaring', 'Natural Gas Flaring', 'CO2-Process Emissions, Methane', 'CO2e Emissions ', 'CCUS Capture Capacity', 'Carbon Prices', 'Oil - Proved reserves', 'Oil - Proved reserves history', 'Oil Production - barrels', 'Oil Production - tonnes', 'Crude+cond production - barrels', 'NGLs production - barrels', 'Liquids Consumption - barrels', 'Oil Consumption - barrels', 'Oil Consumption - Tonnes', 'Oil Consumption - EJ', 'Oil - Regional Consumption', 'Spot crude prices', 'Oil crude prices since 1861', 'Oil refinery - throughput', 'Oil refinery - capacity', 'Oil - Regional refining margins', 'Oil trade movements', 'Oil - Inter-area movements', 'Oil - Trade movements in 22-23', 'Gas - Proved reserves', 'Gas - Proved reserves history ', 'Gas Production - Bcm', 'Gas Production - Bcf', 'Gas Production - EJ', 'Gas Consumption - Bcm', 'Gas Consumption - Bcf', 'Ga

array([['2024 Energy Institute Statistical Review of World Energy'],
       [nan],
       ['This workbook contains information presented in the 2024'],
       ['Energy Institute Statistical Review of World Energy, which can be found on the'],
       ['internet at:'],
       [nan],
       ['EI Statistical Review of World Energy'],
       [nan],
       ['Please use the contents or the tabs at the bottom to navigate between the tables.'],
       [nan],
       ['Primary Energy: Consumption - Exajoules (from 1965)'],
       ['Primary Energy: Consumption by fuel type - Exajoules (2022 and 2023)'],
       ['Primary Energy: Consumption per capita - Gigajoule per capita (from 1965)'],
       [nan],
       ['Carbon: Carbon Dioxide Emissions from Energy (from 1965)'],
       ['Carbon: Carbon Dioxide Emissions from Flaring (from 1975)'],
       ['Natural Gas Flaring'],
       ['Carbon: Carbon Dioxide Equivalent Emissions from Methane and Process Emissions (from 1990)'],
       ['Carbon: Carbon Dio

In [9]:
# Oil Consumption: Barrels (from 1965), in thousand barrels daily
ei_oil_consumption = energy_inst["Oil Consumption - barrels"]

# Set the second row as column headers
ei_oil_consumption.columns = ei_oil_consumption.iloc[1]

# Find duplicate columns
duplicated_cols = ei_oil_consumption.columns[ei_oil_consumption.columns.duplicated()]
print("Duplicated columns:", duplicated_cols)

# Keep the second "2023" column and drop the first one
cols_to_keep = ~ei_oil_consumption.columns.duplicated(keep="first")
ei_oil_consumption = ei_oil_consumption.iloc[:, cols_to_keep]

column_mapping = {
    "Thousand barrels daily": "country_name",
    "2013-23": "growth_rate_per_annum_2013_2023",
}

new_columns = []
for col in ei_oil_consumption.columns:
    # Convert to string
    col_str = str(col)

    # Check if column is in custom mapping
    if col_str in column_mapping:
        new_columns.append(column_mapping[col_str])
    else:
        # Check if the column name can be converted to float and looks like a year
        try:
            if float(col_str) >= 1900 and float(col_str) <= 2100:
                # If it's a year column like '2000.0', convert to '2000'
                new_columns.append(str(int(float(col_str))))
            else:
                # For other numeric columns, convert to snake_case
                new_columns.append(col_str.lower().replace(" ", "_"))
        except (ValueError, TypeError):
            # For non-numeric columns, convert to snake_case
            new_columns.append(col_str.lower().replace(" ", "_").replace("-", "_"))

# Apply the new column names
ei_oil_consumption.columns = new_columns

# Clean up the data
ei_oil_consumption = ei_oil_consumption.replace("..", float("nan"))

# Convert numeric columns to float
numeric_cols = [col for col in new_columns if col != "country_name"]
for col in numeric_cols:
    ei_oil_consumption[col] = pd.to_numeric(ei_oil_consumption[col], errors="coerce")

# Drop rows where all values are NA
ei_oil_consumption = ei_oil_consumption.dropna(how="all")

# Remove the first two rows and cut off at row 104
ei_oil_consumption = ei_oil_consumption.iloc[2:104].copy()

# Reset index after dropping rows
ei_oil_consumption = ei_oil_consumption.reset_index(drop=True)
ei_oil_consumption

Duplicated columns: Index([2023, 2023], dtype='object', name=1)


,country_name,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,growth_rate_per_annum_2013_2023
0,Mexico,315.871088,333.048484,357.305957,386.144762,410.021906,440.735987,467.284852,523.353166,564.177805,...,2037.825767,2085.123858,2038.034090,1918.176846,1817.563961,1602.036162,1769.775415,1931.355326,1961.617607,-0.009963
1,US,11512.436000,12084.373000,12560.345000,13392.866000,14136.795000,14697.186000,15212.493000,16366.984000,17307.679000,...,18499.208493,18592.694872,18845.188693,19416.588252,19423.625252,17183.319352,18784.703315,18862.210266,18983.557337,0.005379
2,Total North America,12936.423389,13584.890278,14163.208026,15101.168112,15927.007234,16609.501741,17191.906098,18479.246401,19626.742450,...,22967.842081,23116.989463,23290.926148,23811.158366,23727.191640,20906.534778,22773.508279,23105.582600,23296.337729,0.003158
3,Argentina,455.874013,471.137797,484.114060,493.543629,516.739504,473.158155,505.365589,501.638934,507.347836,...,695.411056,677.298262,671.546032,644.856482,567.253109,504.314310,612.685196,723.458195,689.692218,0.000639
4,Brazil,305.673808,334.602575,344.540164,412.432377,456.564904,520.116187,587.672022,667.630165,807.227492,...,2583.140145,2453.466355,2485.123066,2367.524169,2361.034320,2218.497517,2394.197835,2512.240799,2566.948105,-0.003217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,Total Asia Pacific,3299.278338,3859.759121,4409.428772,4990.671465,5932.875182,6962.977340,7631.729011,8355.932064,9432.369591,...,32159.586887,33349.487596,34615.403130,35507.648445,36184.117863,34179.876864,35249.882214,36110.480305,38061.126492,0.023253
98,Total World,30884.728725,33315.007646,35641.940695,38529.930312,41968.402228,45676.477412,48180.544118,51869.362347,56064.206201,...,92766.907605,94680.495719,96723.144944,97963.867782,98222.909413,89447.586736,94731.671676,97683.275140,100220.603019,0.010701
99,of which: OECD,23245.355770,25088.253332,26859.695981,29127.938524,31855.624566,34598.225256,36204.385045,38898.854933,41852.228585,...,45295.544370,45848.664864,46456.282788,46759.217162,46474.785138,40886.532788,43807.161193,44769.860359,44731.353647,-0.000652
100,Non-OECD,7639.372955,8226.754314,8782.244714,9401.991788,10112.777662,11078.252156,11976.159073,12970.507414,14211.977616,...,47471.363235,48831.830855,50266.862156,51204.650621,51748.124275,48561.053948,50924.510483,52913.414781,55489.249372,0.020999


In [10]:
# Crude oil & condensate: Production (from 2000), in thousand barrels daily
ei_crude_cond_production = energy_inst["Crude+cond production - barrels"]

# Set the second row as column headers
ei_crude_cond_production.columns = ei_crude_cond_production.iloc[1]

# Find duplicate columns
duplicated_cols = ei_crude_cond_production.columns[
    ei_crude_cond_production.columns.duplicated()
]
print("Duplicated columns:", duplicated_cols)

# Keep the second "2023" column and drop the first one
cols_to_keep = ~ei_crude_cond_production.columns.duplicated(keep="first")
ei_crude_cond_production = ei_crude_cond_production.iloc[:, cols_to_keep]

column_mapping = {
    "Thousand barrels daily": "country_name",
    "2013-23": "growth_rate_per_annum_2013_2023",
}

new_columns = []
for col in ei_crude_cond_production.columns:
    # Convert to string
    col_str = str(col)

    # Check if column is in custom mapping
    if col_str in column_mapping:
        new_columns.append(column_mapping[col_str])
    else:
        # Check if the column name can be converted to float and looks like a year
        try:
            if float(col_str) >= 1900 and float(col_str) <= 2100:
                # If it's a year column like '2000.0', convert to '2000'
                new_columns.append(str(int(float(col_str))))
            else:
                # For other numeric columns, convert to snake_case
                new_columns.append(col_str.lower().replace(" ", "_"))
        except (ValueError, TypeError):
            # For non-numeric columns, convert to snake_case
            new_columns.append(col_str.lower().replace(" ", "_").replace("-", "_"))

# Apply the new column names
ei_crude_cond_production.columns = new_columns

# Clean up the data
ei_crude_cond_production = ei_crude_cond_production.replace("..", float("nan"))

# Convert numeric columns to float
numeric_cols = [col for col in new_columns if col != "country_name"]
for col in numeric_cols:
    ei_crude_cond_production[col] = pd.to_numeric(
        ei_crude_cond_production[col], errors="coerce"
    )

# Drop rows where all values are NA
ei_crude_cond_production = ei_crude_cond_production.dropna(how="all")

# Remove the first two rows and cut off at row 105
ei_crude_cond_production = ei_crude_cond_production.iloc[2:70].copy()

# Reset index after dropping rows
ei_crude_cond_production = ei_crude_cond_production.reset_index(drop=True)
ei_crude_cond_production

Duplicated columns: Index([2023, 2023], dtype='object', name=1)


,country_name,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,growth_rate_per_annum_2013_2023
0,Mexico,3011.965063,3127.036532,3177.098964,3370.896011,3382.898325,3333.348751,3255.578844,3075.709871,2791.575967,...,2266.831389,2153.526317,1948.263211,1833.295181,1700.698132,1705.364434,1756.409759,1784.584521,1875.380805,-0.029194
1,US,5821.604000,5801.401000,5744.078000,5649.300000,5440.968000,5183.822000,5085.864000,5073.900000,4999.671000,...,9439.432000,8846.434000,9357.392000,10951.177000,12311.080000,11318.316000,11267.729000,11910.622000,12927.000000,0.056016
2,Total North America,11029.317934,11150.567354,11281.489621,11515.668946,11387.215108,11036.677655,10996.933734,10899.363484,10506.800578,...,15569.215279,14868.182651,15521.899256,17380.913487,18702.190123,17493.735748,17769.343963,18561.654858,19737.117600,0.038685
3,Argentina,775.868527,812.953364,801.663115,782.814212,733.449060,705.019619,697.917659,689.116081,679.440190,...,532.438815,510.648772,479.996378,489.833044,607.845184,600.996323,682.225787,831.686662,946.155559,0.057698
4,Brazil,1235.000000,1297.000000,1454.000000,1499.000000,1481.000000,1633.575861,1722.729413,1747.992319,1812.224569,...,2437.440349,2509.971215,2621.719965,2586.623252,2787.757735,2939.950377,2905.120916,3021.504455,3401.622738,0.053296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,of which: OECD,18917.790961,18681.458588,18727.482012,18538.695713,18024.608698,17184.501025,16677.125249,16406.981457,15845.390029,...,19945.365771,19094.711591,19636.695153,21497.341444,22853.589355,21708.284075,21810.109519,22436.366503,23641.627349,0.029083
64,Non-OECD,49539.555793,49844.476194,48842.318030,52281.449179,56159.067084,57804.908115,58964.591918,59049.258018,60182.962770,...,61742.749216,62466.108986,62171.673168,62113.641134,60421.087878,55344.705338,56268.879499,58949.128318,59115.541902,-0.001456
65,OPEC,28049.979511,27604.864850,25648.381541,28003.518431,30772.485034,31648.771478,32162.842299,31795.054719,32706.999305,...,32841.342462,33745.620353,33553.366278,33402.276156,31395.740379,27616.029399,28337.951402,30388.118417,29985.249155,-0.004949
66,Non-OPEC,40407.367243,40921.069933,41921.418501,42816.626461,43411.190749,43340.637662,43478.874868,43661.184756,43321.353493,...,48846.772525,47815.200224,48255.002043,50208.706422,51878.936854,49436.960015,49741.037616,50997.376405,52771.920097,0.013342


In [11]:
# Create a unique list of countries from both datasets
countries = pd.DataFrame(
    {
        "country_name": pd.concat(
            [
                ei_crude_cond_production["country_name"],
                ei_oil_consumption["country_name"],
                trade_stag["country_name"],
                trade_optim["country_name"],
                tot_historical["country_name"],
            ]
        ).unique()
    }
)
countries["country_id"] = range(1, len(countries) + 1)

countries

,country_name,country_id
0,Mexico,1
1,US,2
2,Total North America,3
3,Argentina,4
4,Brazil,5
...,...,...
159,Countries with predominantly hydro generation,160
160,Countries with installed capacity below 200 MW,161
161,"Countries with installed capacity above 1,000 MW",162
162,"Countries with installed capacity 200 MW to 1,...",163


In [ ]:
# Step 1: First, merge with the countries table to add country_id
def add_country_ids(df, countries_df):
    # Make sure to retain the original DataFrame columns
    df_with_ids = df.merge(
        countries_df[["country_id", "country_name"]], on="country_name", how="left"
    )
    return df_with_ids


# Let's check the actual column names in your dataframes
print("Production columns:", ei_crude_cond_production.columns.tolist())
print("Countries columns:", countries.columns.tolist())

# Add country_ids to both DataFrames
production_with_ids = add_country_ids(ei_crude_cond_production, countries)
consumption_with_ids = add_country_ids(ei_oil_consumption, countries)

# Now verify that country_id exists in the merged DataFrames
print("Production with IDs columns:", production_with_ids.columns.tolist())

# Step 2: Reshape production data to long format
# Identify year columns (those that are digits)
year_columns_prod = [col for col in production_with_ids.columns if str(col).isdigit()]

# Identify non-year columns to keep as identifiers (exclude country_name as we have country_id now)
id_columns_prod = [
    col
    for col in production_with_ids.columns
    if not str(col).isdigit() and col != "country_name" and col != "country_id"
]

# Now reshape to long format
production_long = pd.melt(
    production_with_ids,
    id_vars=["country_id"]
    + id_columns_prod,  # Include country_id and any other ID columns
    value_vars=year_columns_prod,
    var_name="year",
    value_name="production_barrels",
)

# Convert year to integer type
production_long["year"] = production_long["year"].astype(int)

# Step 3: Reshape consumption data to long format
year_columns_cons = [col for col in consumption_with_ids.columns if str(col).isdigit()]
id_columns_cons = [
    col
    for col in consumption_with_ids.columns
    if not str(col).isdigit() and col != "country_name" and col != "country_id"
]

consumption_long = pd.melt(
    consumption_with_ids,
    id_vars=["country_id"] + id_columns_cons,
    value_vars=year_columns_cons,
    var_name="year",
    value_name="consumption_barrels",
)

# Convert year to integer type
consumption_long["year"] = consumption_long["year"].astype(int)

# Step 4: Clean up the data (handle NaN values, etc.)
production_long = production_long.dropna(subset=["production_barrels"])
consumption_long = consumption_long.dropna(subset=["consumption_barrels"])

# Print the first few rows of each to verify
print("\nProduction Long Format (first 5 rows):")
print(production_long.head())

print("\nConsumption Long Format (first 5 rows):")
print(consumption_long.head())

Production columns: ['country_name', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', 'growth_rate_per_annum_2013_2023']
Countries columns: ['country_name', 'country_id']
Production with IDs columns: ['country_name', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', 'growth_rate_per_annum_2013_2023', 'country_id']

Production Long Format (first 5 rows):
   country_id  growth_rate_per_annum_2013_2023  year  production_barrels
0           1                        -0.029194  2000         3011.965063
1           2                         0.056016  2000         5821.604000
2           3                         0.038685  2000        11029.317934
3           4                         0.057698  2000          775.8

#### Write to csv

In [14]:
production_long.to_csv(
    os.path.join("energy_data", "ei_oil_production.csv"), index=False, sep=","
)
consumption_long.to_csv(
    os.path.join("energy_data", "ei_oil_consumption.csv"), index=False, sep=","
)
countries.to_csv(os.path.join("energy_data", "countries.csv"), index=False, sep=",")

###